# Random Forest

In [1]:
import pandas as pd

credit2 = pd.read_csv('credit2.csv') 

pd.set_option('display.max_columns', None)  # Ajout pour afficher toutes les colonnes

test_size = 0.3

shuffled_data = credit2.sample(frac=1, random_state=42)

split_point = int(len(shuffled_data) * (1 - test_size))

train = shuffled_data.iloc[:split_point].reset_index().rename(columns={'index': 'id'})
test = shuffled_data.iloc[split_point:].reset_index().rename(columns={'index': 'id'})

In [2]:
train

,id,accounts,history_credit,object_credit,savings,employment_old,effort_rate,family_status,guarantees,home_old,property,other_credits,home_status,nb_credits,job_type,nb_of_dependants,telephone,age,duration_credit,amount_credit,presence_unpaid
0,521,CC < 0 euros,A32,Video-HIFI,< 500 euros,between 1 and 4 years,2,Female divorced/separated/married,Without guarantor,2,Property,No external credit,owner,1,A173,1,A191,"[0.0, 25.0)","[15.0, 36.0)","[0.0, 4000.0)",1
1,737,CC < 0 euros,A32,New car,< 500 euros,between 1 and 4 years,3,Male single/married/widowed,Without guarantor,4,Non-property,No external credit,owner,1,A172,2,A192,"[25.0, inf)","[15.0, 36.0)","[4000.0, inf)",0
2,740,CC < 0 euros,A31,New car,< 500 euros,for at least 4 years,2,Male single/married/widowed,Without guarantor,3,Non-property,External credits,owner,1,A173,1,A191,"[25.0, inf)","[15.0, 36.0)","[0.0, 4000.0)",0
3,660,CC > 200 euros,A32,Video-HIFI,< 500 euros,between 1 and 4 years,3,Male single/married/widowed,Without guarantor,4,Property,No external credit,Not owner,1,A173,1,A191,"[0.0, 25.0)","[0.0, 15.0)","[0.0, 4000.0)",0
4,411,No account,A34,Used car,< 500 euros,for at least 4 years,3,Male single/married/widowed,Without guarantor,2,Non-property,No external credit,owner,2,A174,1,A192,"[25.0, inf)","[15.0, 36.0)","[4000.0, inf)",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,262,CC < 0 euros,A34,New car,< 500 euros,for at least 4 years,2,Male single/married/widowed,Without guarantor,4,No property,No external credit,Not owner,3,A174,1,A192,"[25.0, inf)","[15.0, 36.0)","[4000.0, inf)",0
696,610,CC < 0 euros,A32,Interior,< 500 euros,Unemployed or < 1 year,4,Female divorced/separated/married,Without guarantor,3,Not real estate,No external credit,owner,1,A173,1,A191,"[0.0, 25.0)","[0.0, 15.0)","[0.0, 4000.0)",1
697,297,No account,A32,New car,No savings,for at least 4 years,4,Male single/married/widowed,Without guarantor,2,Not real estate,No external credit,owner,1,A172,1,A191,"[25.0, inf)","[0.0, 15.0)","[0.0, 4000.0)",0
698,414,CC < 0 euros,A32,New car,No savings,between 1 and 4 years,4,Female divorced/separated/married,Without guarantor,2,Not real estate,No external credit,owner,1,A173,1,A191,"[25.0, inf)","[15.0, 36.0)","[0.0, 4000.0)",1


In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score

# Pour la reproductibilité
random_state = 235

# CORRECTION: Renommer 'vars' en 'feature_cols' pour éviter de masquer le builtin Python
feature_cols = [
    "accounts", "history_credit", "object_credit", "savings",
    "employment_old", "family_status", "guarantees", "property",
    "other_credits", "home_status", "job_type", "nb_of_dependants",
    "telephone", "duration_credit", "amount_credit"
]    

# Utilisation des données d'entraînement déjà préparées (train/test split)
X_train = train[feature_cols]
y_train = train['presence_unpaid']
X_test = test[feature_cols]
y_test = test['presence_unpaid']

# Instanciation du classifieur RandomForestClassifier
rf = RandomForestClassifier(
    n_estimators=500,
    max_features=3,
    bootstrap=True,
    min_samples_leaf=5,
    random_state=random_state,
    oob_score=True
)

# Entraînement du modèle
rf.fit(X_train, y_train)

# Évaluation sur l'ensemble test
y_pred = rf.predict(X_test)
y_pred_proba = rf.predict_proba(X_test)

# Calcul des métriques
accuracy = accuracy_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_pred_proba[:,1])

# Affichage des résultats
print("Accuracy sur test:", accuracy)
print("AUC sur test:", auc)
print("\nModèle Random Forest entraîné:")
print(rf)
print("\nImportances des variables:")
print(pd.Series(rf.feature_importances_, index=feature_cols).sort_values(ascending=False))
print("\nOOB Score:", rf.oob_score_)

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score

# Pour la reproductibilité
random_state = 235

# Liste des features à utiliser
feature_cols = [
    "accounts", "history_credit", "object_credit", "savings",
    "employment_old", "family_status", "guarantees", "property",
    "other_credits", "home_status", "job_type", "nb_of_dependants",
    "telephone", "duration_credit", "amount_credit"
]

# Utilisation des données d'entraînement déjà préparées (train/test split)
X_train = train[feature_cols]
y_train = train['presence_unpaid']
X_test = test[feature_cols]
y_test = test['presence_unpaid']

# Transformation des features catégorielles en variables numériques
X_train = pd.get_dummies(X_train, drop_first=False)
X_test = pd.get_dummies(X_test, drop_first=False)

# S'assurer que X_test a exactement les mêmes colonnes que X_train
X_train, X_test = X_train.align(X_test, join="outer", axis=1, fill_value=0)

# Instanciation du classifieur RandomForestClassifier
rf = RandomForestClassifier(
    n_estimators=500,
    max_features=3,
    bootstrap=True,
    min_samples_leaf=5,
    random_state=random_state,
    oob_score=True
)

# Entraînement du modèle
rf.fit(X_train, y_train)

# Évaluation sur l'ensemble test
y_pred = rf.predict(X_test)
y_pred_proba = rf.predict_proba(X_test)

# Calcul des métriques
accuracy = accuracy_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_pred_proba[:,1])

# Affichage des résultats
print("Accuracy sur test:", accuracy)
print("AUC sur test:", auc)
print("\nModèle Random Forest entraîné:")
print(rf)
print("\nImportances des variables:")
print(pd.Series(rf.feature_importances_, index=X_train.columns).sort_values(ascending=False))
print("\nOOB Score:", rf.oob_score_)

In [ ]:
import time
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score

# Pour la reproductibilité
random_state = 235
np.random.seed(random_state)

# Liste des features à utiliser
feature_cols = [
    "accounts", "history_credit", "object_credit", "savings",
    "employment_old", "family_status", "guarantees", "property",
    "other_credits", "home_status", "job_type", "nb_of_dependants",
    "telephone", "duration_credit", "amount_credit"
]

# Utilisation des données d'entraînement déjà préparées (train/test split)
X_train = train[feature_cols]
y_train = train['presence_unpaid']
X_test = test[feature_cols]
y_test = test['presence_unpaid']

# Transformation des features catégorielles en variables numériques
X_train = pd.get_dummies(X_train, drop_first=False)
X_test = pd.get_dummies(X_test, drop_first=False)

# Entraînement du modèle Random Forest
cart = RandomForestClassifier(
    n_estimators=500,
    max_features=3,
    bootstrap=True,
    min_samples_leaf=5,
    random_state=random_state,
    oob_score=True
)

# Mesurer le temps d'exécution de l'entraînement
start_time = time.process_time()
cart.fit(X_train, y_train)
elapsed_time = time.process_time() - start_time
print("Temps d'entraînement: {:.2f} secondes".format(elapsed_time))

# Évaluation sur le jeu de test
y_pred = cart.predict(X_test)
y_pred_proba = cart.predict_proba(X_test)
accuracy = accuracy_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_pred_proba[:, 1])

print("Accuracy sur test:", accuracy)
print("AUC sur test:", auc)
print("\nModèle Random Forest entraîné:")
print(cart)
print("\nImportances des variables:")
print(pd.Series(cart.feature_importances_, index=X_train.columns).sort_values(ascending=False))
print("\nOOB Score:", cart.oob_score_)

## Extra-Trees

In [6]:
# Encodage one-hot
X_all = pd.concat([X_train, X_test])
X_all_encoded = pd.get_dummies(X_all, drop_first=False)

# Ré-aligner train et test
X_train_encoded = X_all_encoded.iloc[:len(X_train)]
X_test_encoded = X_all_encoded.iloc[len(X_train):]


In [7]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import roc_auc_score
import time

et = ExtraTreesClassifier(
    n_estimators=1000,
    max_features=5,
    min_samples_leaf=5,
    random_state=235,
    n_jobs=1
)

start_time = time.time()
et.fit(X_train_encoded, y_train)
elapsed_time = time.time() - start_time

# Prédictions de probabilité
y_pred_proba = et.predict_proba(X_test_encoded)[:, 1]

# AUC
auc_score = roc_auc_score(y_test, y_pred_proba)

print("⏱️ Temps d'entraînement :", elapsed_time, "secondes")
print("📊 AUC :", auc_score)


⏱️ Temps d'entraînement : 0.9606852531433105 secondes
📊 AUC : 0.802675585284281


In [8]:
import time
from sklearn.ensemble import ExtraTreesClassifier

def benchmark_extra_trees(n_jobs, n_repeats=5):
    times = []
    for i in range(n_repeats):
        clf = ExtraTreesClassifier(
            n_estimators=500,
            max_features=5,
            min_samples_leaf=5,
            random_state=235 + i,
            n_jobs=n_jobs
        )
        start = time.time()
        clf.fit(X_train_encoded, y_train)
        end = time.time()
        times.append(end - start)
        print(f"Run {i+1}/{n_repeats} (n_jobs={n_jobs}) ⏱️ : {times[-1]:.3f} sec")
    avg_time = sum(times) / n_repeats
    print(f"\n⏱️ Temps moyen (n_jobs={n_jobs}): {avg_time:.3f} sec")
    return times

# Benchmark sans parallélisation
print("=== 🚫 Benchmark n_jobs=1 (pas de parallélisation) ===")
times_1 = benchmark_extra_trees(n_jobs=1)

# Benchmark avec 4 threads
print("\n=== 🧵 Benchmark n_jobs=4 (parallélisation) ===")
times_4 = benchmark_extra_trees(n_jobs=4)


=== 🚫 Benchmark n_jobs=1 (pas de parallélisation) ===
Run 1/5 (n_jobs=1) ⏱️ : 0.470 sec
Run 2/5 (n_jobs=1) ⏱️ : 0.319 sec
Run 3/5 (n_jobs=1) ⏱️ : 0.320 sec
Run 4/5 (n_jobs=1) ⏱️ : 0.320 sec
Run 5/5 (n_jobs=1) ⏱️ : 0.320 sec

⏱️ Temps moyen (n_jobs=1): 0.350 sec

=== 🧵 Benchmark n_jobs=4 (parallélisation) ===
Run 1/5 (n_jobs=4) ⏱️ : 0.335 sec
Run 2/5 (n_jobs=4) ⏱️ : 0.317 sec
Run 3/5 (n_jobs=4) ⏱️ : 0.329 sec
Run 4/5 (n_jobs=4) ⏱️ : 0.320 sec
Run 5/5 (n_jobs=4) ⏱️ : 0.328 sec

⏱️ Temps moyen (n_jobs=4): 0.326 sec


## Forêts aléatoires parallélisées

### Appel Foreach sans parallélisation

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, accuracy_score
import time

# Variables prédictives
feature_cols = [
    "accounts", "history_credit", "object_credit", "savings",
    "employment_old", "family_status", "guarantees", "property",
    "other_credits", "home_status", "job_type", "nb_of_dependants",
    "telephone", "duration_credit", "amount_credit"
]

# Séparation train/test
X_train = train[feature_cols]
y_train = train['presence_unpaid']
X_test = test[feature_cols]
y_test = test['presence_unpaid']

# Encodage indicateur pour les variables catégorielles
X_train = pd.get_dummies(X_train, drop_first=False)
X_test = pd.get_dummies(X_test, drop_first=False)
X_train, X_test = X_train.align(X_test, join="outer", axis=1, fill_value=0)

# Entraînement de 4 modèles de 2500 arbres chacun
estimators = []
start_time = time.time()

for i in range(4):
    rf = RandomForestClassifier(
        n_estimators=2500,
        max_features=3,
        bootstrap=True,
        min_samples_leaf=5,
        random_state=235 + i,
        n_jobs=-1,
        oob_score=True
    )
    rf.fit(X_train, y_train)
    estimators.append(rf)

elapsed_time = time.time() - start_time
print(f"Temps total d'entraînement : {elapsed_time:.2f} secondes")

# Agrégation des probabilités prédictives
y_pred_proba = np.mean([est.predict_proba(X_test)[:,1] for est in estimators], axis=0)
y_pred = (y_pred_proba >= 0.5).astype(int)

# Évaluation
auc = roc_auc_score(y_test, y_pred_proba)
accuracy = accuracy_score(y_test, y_pred)

print("Accuracy :", accuracy)
print("AUC :", auc)

# Importance moyenne des variables
importances = np.mean([est.feature_importances_ for est in estimators], axis=0)
importance_series = pd.Series(importances, index=X_train.columns).sort_values(ascending=False)
print("Importances des variables moyennes :")
print(importance_series)

### Appel Foreach avec parallélisation

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, accuracy_score
from joblib import Parallel, delayed
import time
import multiprocessing

# Détection du nombre de cœurs
num_cores = multiprocessing.cpu_count()

# Liste des features
feature_cols = [
    "accounts", "history_credit", "object_credit", "savings",
    "employment_old", "family_status", "guarantees", "property",
    "other_credits", "home_status", "job_type", "nb_of_dependants",
    "telephone", "duration_credit", "amount_credit"
]

# Données d'entraînement
X_train = pd.get_dummies(train[feature_cols], drop_first=False)
X_test = pd.get_dummies(test[feature_cols], drop_first=False)
X_train, X_test = X_train.align(X_test, join='outer', axis=1, fill_value=0)
y_train = train['presence_unpaid']
y_test = test['presence_unpaid']

# Fonction pour entraîner un sous-modèle RF
def train_rf(seed):
    rf = RandomForestClassifier(
        n_estimators=2500,
        max_features=3,
        bootstrap=True,
        min_samples_leaf=5,
        random_state=seed,
        oob_score=True,
        n_jobs=-1
    )
    rf.fit(X_train, y_train)
    return rf

# Entraînement en parallèle avec joblib
start_time = time.time()
seeds = [235, 236, 237, 238]
estimators = Parallel(n_jobs=4)(delayed(train_rf)(seed) for seed in seeds)
elapsed_time = time.time() - start_time

print(f"Temps total d'entraînement parallèle : {elapsed_time:.2f} secondes")

# Agrégation des prédictions
y_pred_proba = np.mean([rf.predict_proba(X_test)[:, 1] for rf in estimators], axis=0)
y_pred = (y_pred_proba >= 0.5).astype(int)

# Évaluation
auc = roc_auc_score(y_test, y_pred_proba)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy :", accuracy)
print("AUC :", auc)

# Importance des variables moyennes
importances = np.mean([rf.feature_importances_ for rf in estimators], axis=0)
importance_series = pd.Series(importances, index=X_train.columns).sort_values(ascending=False)
print("Importances moyennes des variables :")
print(importance_series.head(10))

### entrainement

In [ ]:
# library
import pandas as pd 
import numpy as np 
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, accuracy_score 
from joblib import Parallel, delayed 
import time 
import multiprocessing 

# détection du nombre de coeurs
num_cores = multiprocessing.cpu_count()

# Liste des features
feature_cols = [
    "accounts", "history_credit", "object_credit", "savings",
    "employment_old", "family_status", "guarantees", "property",
    "other_credits", "home_status", "job_type", "nb_of_dependants",
    "telephone", "duration_credit", "amount_credit"
]

# données d'entrainement 
X_train = pd.get_dummies(train[feature_cols], drop_first=False) 
X_test = pd.get_dummies(test[feature_cols], drop_first=False) 
X_train, X_test = X_train.align(X_test, join='outer', axis=1, fill_value=0)
y_train = train['presence_unpaid'] 
y_test = test['presence_unpaid']

# construction du sous-modele RF
def train_rf(seed):
    rf = RandomForestClassifier(
        n_estimators=2500,
        max_features=3,
        bootstrap=True,
        min_samples_leaf=5,
        random_state=seed,
        oob_score=True,
        n_jobs=-1
    )
    rf.fit(X_train, y_train)
    return rf


# entrainement en parallèle avec joblib
start_time = time.time()
seeds = [235,236,237,238]
estimators = Parallel(n_jobs=4)(delayed(train_rf)(seed) for seed in seeds)
elapsed_time = time.time() - start_time 

print(f"Temps total d'entrainement parallèle : {elapsed_time:.2f} secondes")

# Agrégation des prédictions
y_pred_proba = np.mean([rf.predict_proba(X_test)[:,1] for rf in estimators], axis=0)
y_pred = (y_pred_proba >= 0.5).astype(int)

# Evaluation
auc = roc_auc_score(y_test, y_pred_proba)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy :", accuracy)
print("AUC :", auc)

# importance des variables moyennes
importances = np.mean([rf.feature_importances_ for rf in estimators], axis=0)
importance_series = pd.Series(importances, index=X_train.columns).sort_values(ascending=False)
print("Importances moyennes des variables :")
print(importance_series.head(20))